Курсовая работа


Анализ

In [1]:
from datetime import datetime, timedelta

def generate_schedule(start_time: str, work_duration: int = 12, trip_duration: int = 105, rest_duration: int = 10):
    
    start = datetime.strptime(start_time, "%H:%M")
    end_time = start + timedelta(hours=work_duration)

    current_time = start
    trips = []
    trip_count = 0

    while current_time < end_time:
        
        trip_start = current_time
        trip_end = current_time + timedelta(minutes=trip_duration)

        
        if trip_end > end_time:
            break

        trips.append(f"Trip {trip_count + 1}: {trip_start.strftime('%H:%M')} - {trip_end.strftime('%H:%M')}")
        trip_count += 1
        current_time = trip_end

        
        if trip_count % 2 == 0:
            rest_start = current_time
            rest_end = current_time + timedelta(minutes=rest_duration)

            if rest_end > end_time:
                break

            trips.append(f"Rest: {rest_start.strftime('%H:%M')} - {rest_end.strftime('%H:%M')}")
            current_time = rest_end

    return trips

print("\nDriver 1\n")
schedule = generate_schedule("00:00")
for entry in schedule:
    print(entry)

print("\nDriver 2\n")
schedule = generate_schedule("03:30")
for entry in schedule:
    print(entry)


print("\nDriver 3\n")
schedule = generate_schedule("11:05")
for entry in schedule:
    print(entry)

print("\nDriver 4\n")
schedule = generate_schedule("14:35")
for entry in schedule:
    print(entry)


Driver 1

Trip 1: 00:00 - 01:45
Trip 2: 01:45 - 03:30
Rest: 03:30 - 03:40
Trip 3: 03:40 - 05:25
Trip 4: 05:25 - 07:10
Rest: 07:10 - 07:20
Trip 5: 07:20 - 09:05
Trip 6: 09:05 - 10:50
Rest: 10:50 - 11:00

Driver 2

Trip 1: 03:30 - 05:15
Trip 2: 05:15 - 07:00
Rest: 07:00 - 07:10
Trip 3: 07:10 - 08:55
Trip 4: 08:55 - 10:40
Rest: 10:40 - 10:50
Trip 5: 10:50 - 12:35
Trip 6: 12:35 - 14:20
Rest: 14:20 - 14:30

Driver 3

Trip 1: 11:05 - 12:50
Trip 2: 12:50 - 14:35
Rest: 14:35 - 14:45
Trip 3: 14:45 - 16:30
Trip 4: 16:30 - 18:15
Rest: 18:15 - 18:25
Trip 5: 18:25 - 20:10
Trip 6: 20:10 - 21:55
Rest: 21:55 - 22:05

Driver 4

Trip 1: 14:35 - 16:20
Trip 2: 16:20 - 18:05
Rest: 18:05 - 18:15
Trip 3: 18:15 - 20:00
Trip 4: 20:00 - 21:45
Rest: 21:45 - 21:55
Trip 5: 21:55 - 23:40
Trip 6: 23:40 - 01:25
Rest: 01:25 - 01:35


In [7]:
DRIVERS_COUNT = 8
BUS_COUNT = 10

AVERAGE_BUS_DRIVER_ROUTE_TIME = 90
ROUTE_TIME_DIFFERENCE = 10
ROUTE_TIME_PEAK_HOURS_INCREASES = 5

In [8]:
from datetime import time

class Bus:
    def __init__(self, id: int, available = True):
        self.id = id
        self.available = available

    def is_available(self):
        return self.available


class TimeInterval:
    def __init__(self, start: time, end: time):
        self.start = start
        self.end = end

    def __repr__(self):
        return f"{self.start.strftime('%H:%M')} - {self.end.strftime('%H:%M')}"

    def overlaps(self, other):
        return self.start < other.end and self.end > other.start

class Driver:
    def __init__(self, id: int, type: str):
        self.id = id
        self.type = type
        self.schedule = {
            'Monday': {'work': [], 'breaks': []},
            'Tuesday': {'work': [], 'breaks': []},
            'Wednesday': {'work': [], 'breaks': []},
            'Thursday': {'work': [], 'breaks': []},
            'Friday': {'work': [], 'breaks': []},
            'Saturday': {'work': [], 'breaks': []},
            'Sunday': {'work': [], 'breaks': []}
        }

    def add_work_interval(self, day: str, interval: TimeInterval, bus: Bus):
        self.schedule[day]['work'].append((interval, bus))

    def add_break_interval(self, day: str, interval: TimeInterval):
        self.schedule[day]['breaks'].append(interval)

    def check_overlaps(self, day: str):
        work_intervals = [interval for interval, _ in self.schedule[day]['work']]
        break_intervals = self.schedule[day]['breaks']
        for work in work_intervals:
            for break_time in break_intervals:
                if work.overlaps(break_time):
                    return False
        return True

    def __repr__(self):
        result = f"Driver: {self.id}\n"
        for day, intervals in self.schedule.items():
            result += f"{day}:\n"
            result += "  Work intervals:\n"
            for interval, bus in intervals['work']:
                result += f"    {interval} (Bus: {bus.id})\n"
            result += "  Break intervals:\n"
            for interval in intervals['breaks']:
                result += f"    {interval}\n"
        return result

bus1 = Bus(id=1)
bus2 = Bus(id=2)

# Пример использования
driver1 = Driver(id = 1, type="A")
driver1.add_work_interval('Monday', TimeInterval(start=time(9, 0), end=time(12, 0)), bus=bus1)
driver1.add_work_interval('Monday', TimeInterval(start=time(13, 0), end=time(18, 0)), bus=bus2)
driver1.add_break_interval('Monday', TimeInterval(start=time(12, 0), end=time(13, 0)))

# driver2 = Driver(name="Jane Smith")
# driver2.add_work_interval('Tuesday', TimeInterval(start=time(8, 0), end=time(11, 0)), bus="Bus 3")
# driver2.add_work_interval('Tuesday', TimeInterval(start=time(12, 0), end=time(17, 0)), bus="Bus 4")
# driver2.add_break_interval('Tuesday', TimeInterval(start=time(11, 0), end=time(12, 0)))

print(driver1)
# print(driver2)


Driver: 1
Monday:
  Work intervals:
    09:00 - 12:00 (Bus: 1)
    13:00 - 18:00 (Bus: 2)
  Break intervals:
    12:00 - 13:00
Tuesday:
  Work intervals:
  Break intervals:
Wednesday:
  Work intervals:
  Break intervals:
Thursday:
  Work intervals:
  Break intervals:
Friday:
  Work intervals:
  Break intervals:
Saturday:
  Work intervals:
  Break intervals:
Sunday:
  Work intervals:
  Break intervals:

